<a href="https://colab.research.google.com/github/Ronbragaglia/identificador_bandeira_cartao.ipynb/blob/main/identificador_bandeira_cartao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
from dataclasses import dataclass
from typing import Optional, Dict, Any, List

def luhn_check(number: str) -> bool:
    num = re.sub(r"\D", "", number)
    if not num:
        return False
    digits = list(map(int, num[::-1]))
    total = 0
    for i, d in enumerate(digits):
        if i % 2 == 1:
            dbl = d * 2
            total += dbl - 9 if dbl > 9 else dbl
        else:
            total += d
    return total % 10 == 0

BRAND_PATTERNS = {
    "visa": re.compile(r"^4\d{12}(\d{3})?(\d{3})?$"),
    "mastercard": re.compile(r"^(?:5[1-5]\d{14}|2(?:22[1-9]\d{12}|2[3-9]\d{13}|[3-6]\d{14}|7[11]\d{13}|720\d{12}))$"),
    "amex": re.compile(r"^3\d{13}$"),
    "diners": re.compile(r"^3(?:0[0-5]|\d)\d{11}$"),
    "discover": re.compile(r"^(?:6011\d{12}|65\d{14}|64[4-9]\d{13})$"),
    "jcb": re.compile(r"^(?:2131|1800)\d{11}$|^(?:35\d{3})\d{11}$"),
    "hipercard": re.compile(r"^(?:606282\d{10}(\d{3})?|3841(?:0|4|6)0\d{13})$"),
    "elo": re.compile(
        r"^(?:4011(78|79)|431274|438935|451416|457393|4576(?:3[12]|\d)|504175|627780|636297|636368|636369|"
        r"6503[1-3]|6500(?:3[5-9]|4\d|5[0-1])|6504(?:0[5-9]|[1-3]\d)|650(?:48[5-9]|49\d|50\d|51[1-9]|52\d|53[0-7])|"
        r"6505(?:[4-9]\d)|6507(?:0\d|1[0-8]|2[0-7])|650(?:90[1-9]|91\d|920)|6516(?:5[2-9]|[6-7]\d)|6550(?:0\d|1[1-9]|2[1-9]|[3-4]\d|5[0-8]))\d+$"
    ),
}

SUPPORTED_BRANDS_ORDER = ["visa","mastercard","amex","diners","discover","jcb","hipercard","elo"]

@dataclass
class CardInfo:
    brand: Optional[str]
    valid_luhn: bool
    normalized: str
    length: int

def identify_brand(number: str, validate_luhn: bool = True) -> CardInfo:
    num = re.sub(r"\D", "", number or "")
    brand = None
    for b in SUPPORTED_BRANDS_ORDER:
        if BRAND_PATTERNS[b].match(num):
            brand = b
            break
    valid = luhn_check(num) if validate_luhn and num else False
    return CardInfo(brand=brand, valid_luhn=valid, normalized=num, length=len(num))

def describe_result(info: CardInfo) -> str:
    if not info.normalized:
        return "Entrada vazia/sem dígitos."
    if info.brand and info.valid_luhn:
        return f"Bandeira: {info.brand} | Luhn: válido | Dígitos: {info.length} | Número: {info.normalized}"
    if info.brand and not info.valid_luhn:
        return f"Bandeira: {info.brand} | Luhn: inválido | Dígitos: {info.length} | Número: {info.normalized}"
    if not info.brand and info.valid_luhn:
        return f"Bandeira: indeterminada | Luhn: válido | Dígitos: {info.length} | Número: {info.normalized}"
    return f"Bandeira: indeterminada | Luhn: inválido | Dígitos: {info.length} | Número: {info.normalized}"

def identify_and_print(number: str):
    info = identify_brand(number, validate_luhn=True)
    print(describe_result(info))

TEST_NUMBERS: List[str] = [
    "4111111111111111",
    "4012888888881881",
    "5555555555554444",
    "5105105105105100",
    "378282246310005",
    "371449635398431",
    "30569309025904",
    "6011111111111117",
    "6011000990139424",
    "3530111333300000",
    "3566002020360505",
    "6062825624254001",
    "4011780000000002",
    "4389350000000000",
]

def run_quick_tests():
    print("== Testes rápidos ==")
    for t in TEST_NUMBERS:
        identify_and_print(t)

def write_repo_files(base_dir: str = "."):
    import os, json
    files: Dict[str, str] = {}

    files["src/validator.py"] = (
        "import re\n"
        "from dataclasses import dataclass\n"
        "from typing import Optional\n\n"
        "def luhn_check(number: str) -> bool:\n"
        "    num = re.sub(r\"\\D\", \"\", number)\n"
        "    if not num:\n"
        "        return False\n"
        "    digits = list(map(int, num[::-1]))\n"
        "    total = 0\n"
        "    for i, d in enumerate(digits):\n"
        "        if i % 2 == 1:\n"
        "            dbl = d * 2\n"
        "            total += dbl - 9 if dbl > 9 else dbl\n"
        "        else:\n"
        "            total += d\n"
        "    return total % 10 == 0\n\n"
        "BRAND_PATTERNS = {\n"
        "    \"visa\": re.compile(r\"^4\\d{12}(\\d{3})?(\\d{3})?$\"),\n"
        "    \"mastercard\": re.compile(r\"^(?:5[1-5]\\d{14}|2(?:22[1-9]\\d{12}|2[3-9]\\d{13}|[3-6]\\d{14}|7[11]\\d{13}|720\\d{12}))$\"),\n"
        "    \"amex\": re.compile(r\"^3\\d{13}$\"),\n"
        "    \"diners\": re.compile(r\"^3(?:0[0-5]|\\d)\\d{11}$\"),\n"
        "    \"discover\": re.compile(r\"^(?:6011\\d{12}|65\\d{14}|64[4-9]\\d{13})$\"),\n"
        "    \"jcb\": re.compile(r\"^(?:2131|1800)\\d{11}$|^(?:35\\d{3})\\d{11}$\"),\n"
        "    \"hipercard\": re.compile(r\"^(?:606282\\d{10}(\\d{3})?|3841(?:0|4|6)0\\d{13})$\"),\n"
        "    \"elo\": re.compile(r\"^(?:4011(78|79)|431274|438935|451416|457393|4576(?:3[12]|\\d)|504175|627780|636297|636368|636369|6503[1-3]|6500(?:3[5-9]|4\\d|5[0-1])|6504(?:0[5-9]|[1-3]\\d)|650(?:48[5-9]|49\\d|50\\d|51[1-9]|52\\d|53[0-7])|6505(?:[4-9]\\d)|6507(?:0\\d|1[0-8]|2[0-7])|650(?:90[1-9]|91\\d|920)|6516(?:5[2-9]|[6-7]\\d)|6550(?:0\\d|1[1-9]|2[1-9]|[3-4]\\d|5[0-8]))\\d+$\"),\n"
        "}\n\n"
        "SUPPORTED_BRANDS_ORDER = [\"visa\",\"mastercard\",\"amex\",\"diners\",\"discover\",\"jcb\",\"hipercard\",\"elo\"]\n\n"
        "@dataclass\n"
        "class CardInfo:\n"
        "    brand: Optional[str]\n"
        "    valid_luhn: bool\n"
        "    normalized: str\n"
        "    length: int\n\n"
        "def identify_brand(number: str, validate_luhn: bool = True) -> 'CardInfo':\n"
        "    num = re.sub(r\"\\D\", \"\", number or \"\")\n"
        "    brand = None\n"
        "    for b in SUPPORTED_BRANDS_ORDER:\n"
        "        if BRAND_PATTERNS[b].match(num):\n"
        "            brand = b\n"
        "            break\n"
        "    valid = luhn_check(num) if validate_luhn and num else False\n"
        "    return CardInfo(brand=brand, valid_luhn=valid, normalized=num, length=len(num))\n"
    )

    files["tests/test_validator.py"] = (
        "import unittest\n"
        "from src.validator import identify_brand, luhn_check\n\n"
        "class TestValidator(unittest.TestCase):\n"
        "    def test_luhn(self):\n"
        "        self.assertTrue(luhn_check(\"4111111111111111\"))\n"
        "        self.assertTrue(luhn_check(\"4012888888881881\"))\n"
        "        self.assertTrue(luhn_check(\"5555555555554444\"))\n"
        "        self.assertTrue(luhn_check(\"378282246310005\"))\n"
        "        self.assertFalse(luhn_check(\"4111111111111112\"))\n\n"
        "    def test_brand_detection(self):\n"
        "        self.assertEqual(identify_brand(\"4111 1111 1111 1111\").brand, \"visa\")\n"
        "        self.assertEqual(identify_brand(\"5555555555554444\").brand, \"mastercard\")\n"
        "        self.assertEqual(identify_brand(\"378282246310005\").brand, \"amex\")\n"
        "        self.assertEqual(identify_brand(\"30569309025904\").brand, \"diners\")\n"
        "        self.assertEqual(identify_brand(\"6011111111111117\").brand, \"discover\")\n"
        "        self.assertEqual(identify_brand(\"3530111333300000\").brand, \"jcb\")\n\n"
        "    def test_br_specific(self):\n"
        "        self.assertEqual(identify_brand(\"6062825624254001\").brand, \"hipercard\")\n"
        "        self.assertEqual(identify_brand(\"4011780000000002\").brand, \"elo\")\n\n"
        "if __name__ == \"__main__\":\n"
        "    unittest.main()\n"
    )

    files["README.md"] = (
        "# Identificador de Bandeira de Cartão (Python)\n\n"
        "Aplicativo simples em Python que identifica a bandeira do cartão (Visa, MasterCard, Amex, Diners, Discover, JCB, Elo, Hipercard) a partir do número (IIN/BIN + regex) e valida pelo algoritmo de Luhn.\n"
        "Inclui notebook (Colab), módulo src/validator.py e testes.\n"
    )

    files[".gitignore"] = "__pycache__/\n.ipynb_checkpoints/\n.env\n.venv\n"

    files[".github/workflows/python-ci.yml"] = (
        "name: Python CI\n"
        "on:\n"
        "  push:\n"
        "    branches: [ \"main\" ]\n"
        "  pull_request:\n"
        "    branches: [ \"main\" ]\n"
        "jobs:\n"
        "  test:\n"
        "    runs-on: ubuntu-latest\n"
        "    steps:\n"
        "      - uses: actions/checkout@v4\n"
        "      - uses: actions/setup-python@v5\n"
        "        with:\n"
        "          python-version: \"3.11\"\n"
        "      - run: python -m pip install --upgrade pip\n"
        "      - run: pip install -r requirements.txt || true\n"
        "      - run: python -m unittest -q\n"
    )

    for path, content in files.items():
        full = f"{base_dir}/{path}"
        os.makedirs(os.path.dirname(full), exist_ok=True)
        with open(full, "w", encoding="utf-8") as f:
            f.write(content)
    print("Arquivos do repositório gerados.")

if __name__ == "__main__":
    run_quick_tests()


== Testes rápidos ==
Bandeira: visa | Luhn: válido | Dígitos: 16 | Número: 4111111111111111
Bandeira: visa | Luhn: válido | Dígitos: 16 | Número: 4012888888881881
Bandeira: mastercard | Luhn: válido | Dígitos: 16 | Número: 5555555555554444
Bandeira: mastercard | Luhn: válido | Dígitos: 16 | Número: 5105105105105100
Bandeira: indeterminada | Luhn: válido | Dígitos: 15 | Número: 378282246310005
Bandeira: indeterminada | Luhn: válido | Dígitos: 15 | Número: 371449635398431
Bandeira: amex | Luhn: válido | Dígitos: 14 | Número: 30569309025904
Bandeira: discover | Luhn: válido | Dígitos: 16 | Número: 6011111111111117
Bandeira: discover | Luhn: válido | Dígitos: 16 | Número: 6011000990139424
Bandeira: jcb | Luhn: válido | Dígitos: 16 | Número: 3530111333300000
Bandeira: jcb | Luhn: válido | Dígitos: 16 | Número: 3566002020360505
Bandeira: hipercard | Luhn: válido | Dígitos: 16 | Número: 6062825624254001
Bandeira: visa | Luhn: inválido | Dígitos: 16 | Número: 4011780000000002
Bandeira: visa | 